In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 15.5 MB/s eta 0:00:00


Importing the libraries

In [2]:
from transformers import BertTokenizer, BertForMaskedLM
from transformers import LineByLineTextDataset
from transformers import Trainer, TrainingArguments

Load Polish Bert Tokenizer

In [3]:
tokenizer = BertTokenizer.from_pretrained('dkleczek/bert-base-polish-uncased-v1', from_tf=True)

model loading

In [12]:
model = BertForMaskedLM.from_pretrained('dkleczek/bert-base-polish-uncased-v1')

Some weights of the model checkpoint at dkleczek/bert-base-polish-uncased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Data loading and pre-processing

In [4]:
max_length = 0
with open('/content/dataset.txt', 'r', encoding='utf-8') as f:
    for line in f:
        length = len(line.strip())
        if length > max_length:
            max_length = length
            


Checking seq length

In [5]:
with open('/content/dataset.txt', 'r', encoding='utf-8') as f:
    for line in f:
        tokens = tokenizer(line.strip(), padding='max_length', truncation=True, max_length=128)
        print(len(tokens['input_ids']))

Streaming output truncated to the last 5000 lines.
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
1

creating dataset

In [6]:
dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="/content/dataset.txt",
    block_size=max_length, # Set the block_size to 128
)

/usr/local/lib/python3.9/dist-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


training args

In [7]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    save_total_limit=2,
    learning_rate=2e-5,
    warmup_steps=500,
    logging_dir='./logs',
    logging_steps=500,
    save_strategy='epoch',
)

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


training

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)


saving the model

In [14]:
model.save_pretrained('./fine_tuned_bert_model')